In [2]:
import torch
from models.components.ae import SimpleConvEncoder, SimpleConvDecoder

In [3]:
# Set your config values
in_dim = 2
latent_dim = 64
levels = 2
min_ch = 16
ch_mult = 4
out_dim = 2

# Dummy input: batch size 1, 2 channels, 240x370
x = torch.randn(1, in_dim, 240, 370)

# Encoder
encoder = SimpleConvEncoder(in_dim=in_dim, levels=levels, min_ch=min_ch, ch_mult=ch_mult)
h = encoder(x)
print("Encoder output shape:", h.shape)

# Project to latent mean/logvar
to_moments = torch.nn.Conv2d(h.shape[1], 2*latent_dim, 1)
moments = to_moments(h)
mean, logvar = torch.chunk(moments, 2, dim=1)
print("Latent mean shape:", mean.shape)

# Decoder
decoder = SimpleConvDecoder(in_dim=latent_dim, levels=levels, min_ch=min_ch, out_dim=out_dim)
z = torch.randn(1, latent_dim, h.shape[2], h.shape[3])
out = decoder(z)
print("Decoder output shape:", out.shape)

normalization called with channels=2, norm_type=group, norm_kwargs={'num_groups': 1}
normalization called with channels=16, norm_type=group, norm_kwargs={'num_groups': 1}
normalization called with channels=16, norm_type=group, norm_kwargs={'num_groups': 1}
normalization called with channels=32, norm_type=group, norm_kwargs={'num_groups': 1}
Encoder output shape: torch.Size([1, 32, 60, 92])
Latent mean shape: torch.Size([1, 64, 60, 92])
normalization called with channels=256, norm_type=group, norm_kwargs={'num_groups': 1}
normalization called with channels=256, norm_type=group, norm_kwargs={'num_groups': 1}
normalization called with channels=2, norm_type=group, norm_kwargs={'num_groups': 1}
normalization called with channels=2, norm_type=group, norm_kwargs={'num_groups': 1}
Decoder output shape: torch.Size([1, 2, 240, 368])
